In [16]:
import pandas as pd
import json
import re

# map the title of books and the reviews

In [64]:
# extract the title of books
bk_2019 = pd.read_csv("../data/raw_data/books_2019_df.csv")
title_2019 = bk_2019['title'].apply(lambda x: x.lower())
title_df_2019 = pd.DataFrame(title_2019,columns=['title'])
title_df_2019['title_id'] = ['Book_' + str(i) for i in range(1, len(title_df_2019) + 1)]
title_df_2019.head(3)

,title,title_id
0,the testaments,Book_1
1,normal people,Book_2
2,where the forest meets the stars,Book_3


In [73]:
bk_2020 = pd.read_csv("../data/raw_data/books_2020_df.csv")
title_2020 = bk_2020['title'].apply(lambda x: x.lower())
title_df_2020 = pd.DataFrame(title_2020,columns=['title'])
title_df_2020['title_id'] = ['Book_' + str(i) for i in range(1, len(title_df_2020) + 1)]
title_df_2020.head(3)

,title,title_id
0,the midnight library,Book_1
1,anxious people,Book_2
2,american dirt,Book_3


In [74]:
bk_2021 = pd.read_csv("../data/raw_data/books_2021_df.csv")
title_2021 = bk_2021['title'].apply(lambda x: x.lower())
title_df_2021 = pd.DataFrame(title_2021,columns=['title'])
title_df_2021['title_id'] = ['Book_' + str(i) for i in range(1, len(title_df_2021) + 1)]
title_df_2021.head(3)

,title,title_id
0,"beautiful world, where are you",Book_1
1,dial a for aunties,Book_2
2,cloud cuckoo land,Book_3


In [75]:
bk_2022 = pd.read_csv("../data/raw_data/books_2022_df.csv")
title_2022 = bk_2022['title'].apply(lambda x: x.lower())
title_df_2022 = pd.DataFrame(title_2022,columns=['title'])
title_df_2022['title_id'] = ['Book_' + str(i) for i in range(1, len(title_df_2022) + 1)]
title_df_2022.head(3)

,title,title_id
0,"tomorrow, and tomorrow, and tomorrow",Book_1
1,wish you were here,Book_2
2,the winners,Book_3


In [76]:
bk_2023 = pd.read_csv("../data/raw_data/books_2023_df.csv")
title_2023 = bk_2023['title'].apply(lambda x: x.lower())
title_df_2023 = pd.DataFrame(title_2023,columns=['title'])
title_df_2023['title_id'] = ['Book_' + str(i) for i in range(1, len(title_df_2023) + 1)]
title_df_2023.head(3)

,title,title_id
0,yellowface,Book_1
1,hello beautiful,Book_2
2,the wishing game,Book_3


In [72]:
with open("../data/raw_data/reviews_2019.json", 'r',encoding='utf-8') as file:
    review_2019 = json.load(file)

with open("../data/raw_data/reviews_2020.json", 'r',encoding='utf-8') as file:
    review_2020 = json.load(file)

with open("../data/raw_data/reviews_2021.json", 'r',encoding='utf-8') as file:
    review_2021 = json.load(file)

with open("../data/raw_data/reviews_2022.json", 'r',encoding='utf-8') as file:
    review_2022 = json.load(file)

with open("../data/raw_data/reviews_2023.json", 'r',encoding='utf-8') as file:
    review_2023 = json.load(file)

In [50]:
# define function 

def review_to_df(data):
    reviews = []

    # Iterate through each book and its reviews
    for book_id, reviews_dict in data.items():
        for review_id, review_content in reviews_dict.items():
            # Extracting the reviewer's name and review text
            # Here we assume the name ends at the first digit, which is extremely simplistic
            reviewer_name = ''.join([char for char in review_content if not char.isdigit()]).split(',')[0]
            review_text = review_content.split('followers')[-1]  # This is a simplistic split

            # Append the extracted information as a dictionary to the list
            reviews.append({
                'title_id': book_id,
                'review_id': review_id,
                'reviewer_name': reviewer_name.strip(),
                'review_text': review_text.strip()
            })
    df_reviews = pd.DataFrame(reviews)
    return df_reviews

In [77]:
review_df_2019 = review_to_df(review_2019)
review_df_2019.shape

(3980, 4)

In [79]:
review_df_2020 = review_to_df(review_2020)
review_df_2021 = review_to_df(review_2021)
review_df_2022 = review_to_df(review_2022)
review_df_2023 = review_to_df(review_2023)

In [60]:
# Regular expression to match the date format
# Define the date pattern
date_pattern = r'(January|February|March|April|May|June|July|August|September|October|November|December)\s\d{1,2},\s\d{4}'

# Function to extract and remove dates
def extract_and_remove_date(text):
    match = re.search(date_pattern, text)
    if match:
        date = match.group(0)  # Extract the date
        cleaned_text = re.sub(date_pattern, '', text)  # Remove the date from the text
        return date, cleaned_text
    return None, text  # Return None for date and original text if no date found

In [61]:
# Apply the function to each entry in the review_text column
review_df_2019[['date', 'review_text']] = review_df_2019['review_text'].apply(
    lambda x: pd.Series(extract_and_remove_date(x))
)
df_2019 = pd.merge(title_df_2019, review_df_2019, on='title_id', how='left')
df_2019 = df_2019[['title','review_id','reviewer_name','date','review_text']]
df_2019

,title_id,review_id,reviewer_name,review_text,date
0,Book_1,1,Tatiana,I guess I'll have to be the one who says what ...,"May 29, 2020"
1,Book_1,2,Emily May,I can sum it up simply:this book is not needed...,"September 13, 2019"
2,Book_1,3,Marchpane reviews,Return to GileadCheck your expectations at the...,"January 22, 2020"
3,Book_1,4,Nilufer Ozmekik,Winner of best fiction category but it’s not m...,"August 15, 2021"
4,Book_1,5,Sean Barrs,A review in 5 words:Unnecessary. Pointless. Ru...,"December 4, 2019"
...,...,...,...,...,...
3975,Book_400,6,Jennifer,A bi-racial family enjoys cozy-ing up together...,"November 15, 2019"
3976,Book_400,7,Amy Layton,With the current state of the world where we'r...,"January 27, 2020"
3977,Book_400,8,Tracy reviews followersFebruary,2019 Goodreads Choice Awards Picture Book Nomi...,"February 21, 2020"
3978,Book_400,9,Stacie,"Stacie2,030 reviewsThe Whos shown in Whoville ...","December 17, 2019"


In [80]:
review_df_2020[['date', 'review_text']] = review_df_2020['review_text'].apply(
    lambda x: pd.Series(extract_and_remove_date(x))
)
df_2020 = pd.merge(title_df_2020, review_df_2020, on='title_id', how='left')
df_2020 = df_2020[['title','review_id','reviewer_name','date','review_text']]
df_2020

,title_id,review_id,reviewer_name,review_text,date
0,Book_1,1,Nataliya reviews.k followersJanuary,I liked this book until it suddenly decided to...,"January 21, 2022"
1,Book_1,2,Gabby,"“The only way to learn is to live”Fuck, this b...","November 24, 2020"
2,Book_1,3,Nicole reviews,Everybody probably knows the premise of this b...,"February 15, 2023"
3,Book_1,4,Nilufer Ozmekik,Okay! No more words! This is one of the best s...,"September 14, 2023"
4,Book_1,5,Paromjit,It is no secret that Matt Haig has mental heal...,"May 31, 2020"
...,...,...,...,...,...
3965,Book_399,6,Ms. B,What a fun way to interest young ones (and the...,"November 15, 2020"
3966,Book_399,7,Coleccionista de finales tristes reviews follo...,Dan Brown es conocido por “El código Da vinci”...,"November 12, 2020"
3967,Book_399,8,Dana Al-Basha | دانة الباشا,"I liked this book a lot... very interactive, w...","October 11, 2021"
3968,Book_399,9,Lina reviews followersJuly,Labai nuostabios iliustracijos. Tokios-mano mė...,"July 22, 2022"


In [82]:
review_df_2021[['date', 'review_text']] = review_df_2021['review_text'].apply(
    lambda x: pd.Series(extract_and_remove_date(x))
)
df_2021 = pd.merge(title_df_2021, review_df_2021, on='title_id', how='left')
df_2021 = df_2021[['title','review_id','reviewer_name','date','review_text']]
df_2021

,title,review_id,reviewer_name,date,review_text
0,"beautiful world, where are you",1,Nataliya reviews.k followersAugust,"August 10, 2022",“Purposeful awkwardness” is how I can describe...
1,"beautiful world, where are you",2,Thomas,"September 11, 2021",Let me start this review by saying that I thin...
2,"beautiful world, where are you",3,emma,"December 14, 2022","Well, well, well.Look what the cat dragged in...."
3,"beautiful world, where are you",4,Tim Koh (on hiatus) reviews followersSeptember,"September 14, 2021",I am writing a Sally Rooney novel. I am typing...
4,"beautiful world, where are you",5,Emily May,"September 16, 2021",I don't know what it is.All of you scratching ...
...,...,...,...,...,...
3377,where's my joey?,6,Amy reviews followersDecember,"December 31, 2020",I won this through Goodread's FirstReads givea...
3378,where's my joey?,7,Sam Andersen reviews followersSeptember,"September 22, 2021",This is a sweet adorable book that teaches chi...
3379,where's my joey?,8,Andrea Pole reviews followersJanuary,"January 27, 2021",Where's My Joey? by Wendy Monica Winter is a c...
3380,where's my joey?,9,Robbie reviews followersMarch,"March 9, 2021",Where’s My Joey? by Wendy Monica Winter is a c...


In [83]:
review_df_2022[['date', 'review_text']] = review_df_2022['review_text'].apply(
    lambda x: pd.Series(extract_and_remove_date(x))
)
df_2022 = pd.merge(title_df_2022, review_df_2022, on='title_id', how='left')
df_2022 = df_2022[['title','review_id','reviewer_name','date','review_text']]
df_2022

,title,review_id,reviewer_name,date,review_text
0,"tomorrow, and tomorrow, and tomorrow",1,Katherine reviewsJune,"June 25, 2022",Katherine337 reviewsThis book is so utterly pr...
1,"tomorrow, and tomorrow, and tomorrow",2,Regina,"July 16, 2022","Tomorrow, and Tomorrow, and Tomorrow I will st..."
2,"tomorrow, and tomorrow, and tomorrow",3,Val ⚓️ Shameless Handmaiden ⚓️,"December 14, 2022",***5 Stars***This was one of those books where...
3,"tomorrow, and tomorrow, and tomorrow",4,Meredith (Trying to catch up!) reviews.k follo...,"May 11, 2022","Complicated and MemorableTomorrow, and Tomorro..."
4,"tomorrow, and tomorrow, and tomorrow",5,Emily May,"July 13, 2022",I feel... really disappointed by this. I've lo...
...,...,...,...,...,...
3377,how to heal a gryphon,6,Ms. B,"September 25, 2023",Remember the classicStrega NonabyTomie dePaola...
3378,how to heal a gryphon,7,Jaime reviews followersSeptember,"September 26, 2022",How To Heal A Gryphon was a pretty good Middle...
3379,how to heal a gryphon,8,Cathy reviews followersMarch,"March 28, 2022",With Christmas Eve rapidly approaching where 1...
3380,how to heal a gryphon,9,Rachael reviewsSeptember,"September 23, 2022",Rachael173 reviews3.5 stars rounded upGiada Be...


In [85]:
review_df_2023[['date', 'review_text']] = review_df_2023['review_text'].apply(
    lambda x: pd.Series(extract_and_remove_date(x))
)
df_2023 = pd.merge(title_df_2023, review_df_2023, on='title_id', how='left')
df_2023 = df_2023[['title','review_id','reviewer_name','date','review_text']]
df_2023

,title,review_id,reviewer_name,date,review_text
0,yellowface,1,ishika reviews followersJanuary,None,2.5/5i feel weird writing this review. for one...
1,yellowface,2,s.penkevich,None,‘Don’t ghosts just want to be remembered?’A bo...
2,yellowface,3,idiomatic reviews followersJanuary,None,"lmao.the frustration, as always, is that rf ku..."
3,yellowface,4,Bookishrealm,None,Whew child. Some of these reviews are doing ex...
4,yellowface,5,Emily May,None,"If publishing is rigged, you might as well mak..."
...,...,...,...,...,...
2985,"not funny: essays on life, comedy, culture, et...",6,Hein Matthew Hattie reviews followersNovember,None,DNF / abandoned p 76. Average comedian memoir-...
2986,"not funny: essays on life, comedy, culture, et...",7,Richard PropesAuthor books followersDecember,None,If you pick up Academy Award nominee Jena Frie...
2987,"not funny: essays on life, comedy, culture, et...",8,Sharon L reviewsMay,None,Sharon L5 reviewsIf she was trying to be funny...
2988,"not funny: essays on life, comedy, culture, et...",9,Laura Hobbs reviewsAugust,None,Laura Hobbs18 reviewsNot funny. While I starte...


In [95]:
df = pd.concat([df_2019,df_2020,df_2021,df_2022,df_2023], ignore_index=True)
df = df.drop(columns=['reviewer_name','date'])
df.head(3)

,title,review_id,review_text
0,the testaments,1,I guess I'll have to be the one who says what ...
1,the testaments,2,I can sum it up simply:this book is not needed...
2,the testaments,3,Return to GileadCheck your expectations at the...


In [96]:
# export as csv file
df.to_csv('../data/database/reviews.csv', index=False)

In [97]:
df.isna().sum()

title          0
review_id      8
review_text    8
dtype: int64